In [105]:
import mne
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
from glob import glob
import os

# Reading Data

In [106]:
# Define the relative path pattern for .edf files in all subdirectories of the Data directory
path_pattern = os.path.join('..', '..', 'CleanedEpochs', 'Baseline', 'Subj*', '*.fif')
# Use glob to get all matching files
epoch_files = glob(path_pattern, recursive=True)

# Print the list of .fif files
for e in epoch_files:
    print(e)

..\..\CleanedEpochs\Baseline\Subj1\Fast-epo.fif
..\..\CleanedEpochs\Baseline\Subj1\Slow-epo.fif
..\..\CleanedEpochs\Baseline\Subj10\Fast-epo.fif
..\..\CleanedEpochs\Baseline\Subj10\Slow-epo.fif
..\..\CleanedEpochs\Baseline\Subj11\Fast-epo.fif
..\..\CleanedEpochs\Baseline\Subj11\Slow-epo.fif
..\..\CleanedEpochs\Baseline\Subj12\Fast-epo.fif
..\..\CleanedEpochs\Baseline\Subj12\Slow-epo.fif
..\..\CleanedEpochs\Baseline\Subj13\Fast-epo.fif
..\..\CleanedEpochs\Baseline\Subj13\Slow-epo.fif
..\..\CleanedEpochs\Baseline\Subj14\Fast-epo.fif
..\..\CleanedEpochs\Baseline\Subj14\Slow-epo.fif
..\..\CleanedEpochs\Baseline\Subj15\Fast-epo.fif
..\..\CleanedEpochs\Baseline\Subj15\Slow-epo.fif
..\..\CleanedEpochs\Baseline\Subj16\Fast-epo.fif
..\..\CleanedEpochs\Baseline\Subj16\Slow-epo.fif
..\..\CleanedEpochs\Baseline\Subj2\Fast-epo.fif
..\..\CleanedEpochs\Baseline\Subj2\Slow-epo.fif
..\..\CleanedEpochs\Baseline\Subj3\Fast-epo.fif
..\..\CleanedEpochs\Baseline\Subj3\Slow-epo.fif
..\..\CleanedEpochs\Baseli

In [107]:
md = pd.read_csv('../../baseline_metadata.csv')
epochs = [mne.read_epochs(f) for f in epoch_files]

Reading e:\Campus\RM\FinalProject\Codes\FeatureExtraction\..\..\CleanedEpochs\Baseline\Subj1\Fast-epo.fif ...
    Found the data of interest:
        t =       0.00 ...    1992.19 ms
        0 CTF compensation matrices available
Not setting metadata
51 matching events found
No baseline correction applied
0 projection items activated
Reading e:\Campus\RM\FinalProject\Codes\FeatureExtraction\..\..\CleanedEpochs\Baseline\Subj1\Slow-epo.fif ...
    Found the data of interest:
        t =       0.00 ...    1992.19 ms
        0 CTF compensation matrices available
Not setting metadata
57 matching events found
No baseline correction applied
0 projection items activated
Reading e:\Campus\RM\FinalProject\Codes\FeatureExtraction\..\..\CleanedEpochs\Baseline\Subj10\Fast-epo.fif ...
    Found the data of interest:
        t =       0.00 ...    1992.19 ms
        0 CTF compensation matrices available
Not setting metadata
44 matching events found
No baseline correction applied
0 projection items acti

# Feature Extraction using PSD

In [108]:
res_df = pd.DataFrame()

In [109]:
FREQ_BANDS = {
    "delta": (1, 4),
    "theta": (4, 8),
    "alpha": (8, 12),
    "beta": (12, 30),
    "gamma": (30, 50),
}

# Define brain regions and their corresponding channel names
BRAIN_REGIONS = {
    "frontal": ["AF3", "F7", "F3", "FC5", "FC6", "F4", "F8", "AF4"],
    "temporal": ["T7", "T8"],
    "parietal": ["P7", "P8"],
    "occipital": ["O1", "O2"],
}

# Get the channel names from the first epochs object
channels = epochs[0].ch_names

# Extract indices for each brain region
region_indices = {
    region: [channels.index(ch) for ch in ch_names if ch in channels]
    for region, ch_names in BRAIN_REGIONS.items()
}

In [110]:
# Define the function to compute power bands
from mne.time_frequency import psd_array_welch
def get_power_band(epochs, region_indices):
    """
    Input: epochs object
    Output: averaged power band for each brain region
    Output shape: (5, 4) -> 5 frequency bands and 4 brain regions
    """
    sfreq = epochs.info['sfreq']  # Sampling frequency
    epoch_length = epochs.times[-1] - epochs.times[0]  # Duration of each epoch in seconds

    # Set `n_per_seg` to match the duration of each epoch
    n_per_seg = int(sfreq * epoch_length)

    # Set `n_fft` to match `n_per_seg` for computational efficiency
    n_fft = n_per_seg

    # Compute PSD using Welch's method
    psds, freqs = psd_array_welch(epochs.get_data(copy=False), sfreq=sfreq, n_fft=n_fft, n_per_seg=n_per_seg, n_overlap=n_per_seg // 2)
    psds *= 1e12  # Convert to uV^2/Hz
    psds = 10 * np.log10(psds)  # Convert to dB

    # Average over epochs
    psds = psds.mean(axis=0)

    psds_per_region = []
    for fmin, fmax in FREQ_BANDS.values():
        psds_band = psds[:, (freqs >= fmin) & (freqs < fmax)].mean(axis=-1)
        psds_per_region.append(
            [psds_band[region_indices[region]].mean() for region in BRAIN_REGIONS]
        )

    return np.array(psds_per_region)

In [111]:
# Compute power bands and add to DataFrame
for i, epoch in enumerate(epochs):
    psds_per_region = get_power_band(epoch, region_indices)

    # Flatten the power band data
    flattened_data = psds_per_region.flatten()

    # Create column names for the flattened data
    column_names = [
        f"{freq_band}_{region}"
        for freq_band in FREQ_BANDS.keys()
        for region in BRAIN_REGIONS.keys()
    ]

    # Add the flattened data to the DataFrame
    for j, col_name in enumerate(column_names):
        res_df.loc[i, col_name] = flattened_data[j]

Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective wind

In [112]:
res_df

,delta_frontal,delta_temporal,delta_parietal,delta_occipital,theta_frontal,theta_temporal,theta_parietal,theta_occipital,alpha_frontal,alpha_temporal,alpha_parietal,alpha_occipital,beta_frontal,beta_temporal,beta_parietal,beta_occipital,gamma_frontal,gamma_temporal,gamma_parietal,gamma_occipital
0,15.076819,13.143917,13.041501,14.512219,5.362843,2.586964,3.260040,4.885585,0.912455,-2.322239,-1.989005,0.671899,-3.041057,-7.069346,-6.625545,-3.648806,-9.290189,-12.706184,-12.420840,-11.448307
1,7.984208,12.388075,9.472491,10.179127,2.392743,3.269588,3.419422,3.750249,1.616374,1.160028,3.267333,5.936005,-3.304798,-5.292544,-3.792953,-2.536451,-10.020468,-12.482489,-11.772631,-11.564416
2,11.239703,8.465389,10.528402,10.836222,2.659395,0.370493,2.008323,3.089041,-2.185492,-3.879035,-3.230922,-0.143173,-5.354958,-6.251469,-6.002989,-2.923501,-11.131412,-11.450147,-11.624944,-9.015243
3,4.745000,4.652828,5.134093,7.363774,-0.173816,-0.365668,-2.079406,1.302277,-3.315534,-2.989607,-5.012758,0.277222,-5.882658,-4.621961,-7.396070,-2.362242,-11.521375,-9.694671,-12.043571,-9.376422
4,6.563949,5.086261,5.107025,6.722253,-1.419835,-2.329013,-1.127842,-1.234357,-2.960157,-1.689096,-2.361830,0.069344,-5.920830,-2.084703,-5.506969,-4.850824,-12.390167,-7.897711,-13.358566,-14.384893
5,9.105438,7.729108,9.245042,9.166969,0.209889,-1.640848,-0.355593,-0.538688,-2.866510,-5.344317,-2.873485,-0.780645,-6.666129,-7.848193,-6.825153,-6.212503,-12.600817,-12.997016,-13.446814,-13.747684
6,8.900696,5.972916,6.561369,5.544251,-0.153909,-1.652258,-1.671633,-1.230073,2.295218,3.055246,3.418385,7.012262,-6.569969,-3.864689,-6.712262,-4.643357,-13.043025,-9.782082,-13.742042,-12.275228
7,6.003339,4.097141,4.936740,6.308200,-0.076791,-1.042578,-0.088724,0.061239,3.779697,3.338430,7.975719,8.848449,-6.133783,-2.689306,-4.107778,-3.582231,-12.898985,-9.039561,-12.079457,-11.710881
8,9.192189,8.992609,8.756006,10.714357,1.232150,0.776942,1.167790,2.773153,-3.110295,-2.335046,-1.507006,0.296135,-4.586285,-2.915835,-4.096866,-2.720909,-10.888431,-8.802869,-10.499598,-9.906814
9,10.641555,11.497464,12.529074,13.069721,2.588283,3.390289,4.278135,3.903502,-1.939123,-0.253894,-0.848007,0.826779,-5.253194,-1.237171,-3.241757,-3.700632,-12.230278,-8.800054,-10.883303,-11.690459


In [114]:
# Concat with subject_id and pace
res_df.to_csv('../../baseline_band_powers.csv', index=False)